# RAG Implementation with Anthropic Claude and OpenAI

This notebook demonstrates a complete Retrieval-Augmented Generation (RAG) pipeline using:
- **OpenAI** for semantic embeddings (Ada-002)
- **Anthropic Claude** for generation (Haiku/Sonnet/Opus)
- **ChromaDB** for local vector storage

A production-ready implementation for building AI-powered search and Q&A systems.


In [ ]:
# Setup - Install required packages
# Using UV for fast package management (10-100x faster than pip)

# Run this in terminal:
# .\setup_uv.ps1  # Windows
# ./setup_uv.sh    # Linux/Mac

# Or install with uv:
# uv pip install anthropic openai chromadb python-dotenv pandas numpy scikit-learn tenacity


In [ ]:
import os
import sys
from pathlib import Path

# Add src to path so we can import our modules
sys.path.append(str(Path.cwd()))

# Import required libraries
import anthropic
import openai
import chromadb
import numpy as np
import pandas as pd
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Import our RAG modules
from src.rag_pipeline import RAGPipeline
from src.embeddings import OpenAIEmbeddings
from src.vector_store import VectorStore
from src.chunking import TextChunker, MarkdownChunker

print("✅ All imports successful!")
print(f"Anthropic version: {anthropic.__version__}")
print(f"OpenAI version: {openai.__version__}")
print(f"Current directory: {Path.cwd()}")


## Step 1: Configure API Keys

Make sure you have set the following environment variables in your `.env` file:
- `ANTHROPIC_API_KEY` - For Claude text generation
- `OPENAI_API_KEY` - For embeddings


In [ ]:
# Check if API keys are configured
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
openai_key = os.getenv("OPENAI_API_KEY")

if not anthropic_key:
    print("❌ ANTHROPIC_API_KEY not found in environment variables")
    print("   Get your key at: https://console.anthropic.com/")
else:
    print(f"✅ Anthropic API key configured: {anthropic_key[:10]}...")

if not openai_key:
    print("❌ OPENAI_API_KEY not found in environment variables")
    print("   Get your key at: https://platform.openai.com/")
else:
    print(f"✅ OpenAI API key configured: {openai_key[:10]}...")


## Step 2: Initialize RAG Pipeline

We'll create a RAG pipeline with:
- Claude 3 Haiku for fast, cost-effective generation
- OpenAI Ada-002 embeddings for semantic search (1536 dimensions)
- ChromaDB for local vector storage


In [ ]:
# Initialize the RAG pipeline
rag = RAGPipeline(
    model="claude-3-haiku-20240307",  # Fast and cost-effective
    collection_name="demo_collection",
    chunk_size=512,  # Characters per chunk
    chunk_overlap=50  # Overlap between chunks
)

# Clear any existing data
rag.clear_knowledge_base()

print("✅ RAG pipeline initialized!")
print(f"📊 Stats: {rag.get_stats()}")


## Step 3: Add Documents to Knowledge Base

Let's add some sample documents about AI and machine learning.


In [ ]:
# Sample documents
documents = [
    """
    # Large Language Models (LLMs)
    
    Large Language Models are neural networks trained on vast amounts of text data.
    They can understand and generate human-like text across many domains.
    
    ## Key Capabilities
    - Text generation and completion
    - Question answering
    - Summarization
    - Translation
    - Code generation
    
    ## Popular Models
    - GPT-4 (OpenAI)
    - Claude (Anthropic)
    - PaLM (Google)
    - LLaMA (Meta)
    """,
    
    """
    # Embedding Models
    
    Embedding models convert text into dense vector representations that capture
    semantic meaning. These vectors enable semantic search and similarity comparisons.
    
    ## How Embeddings Work
    1. Text is tokenized into smaller units
    2. Tokens are processed through neural networks
    3. Output is a fixed-size vector (e.g., 1024 dimensions)
    4. Similar texts have similar vectors
    
    ## Applications
    - Semantic search
    - Recommendation systems
    - Clustering and classification
    - RAG systems
    """,
    
    """
    # Vector Databases
    
    Vector databases are specialized databases designed to store and search
    high-dimensional vectors efficiently using similarity metrics.
    
    ## Popular Vector Databases
    - Pinecone: Fully managed, cloud-native
    - ChromaDB: Open-source, embedded
    - Weaviate: Open-source with hybrid search
    - Qdrant: Open-source with rich filtering
    - FAISS: Facebook's similarity search library
    
    ## Key Features
    - Fast similarity search
    - Scalability to billions of vectors
    - Metadata filtering
    - Hybrid search capabilities
    """
]

# Add documents with metadata
metadatas = [
    {"title": "Large Language Models", "category": "AI Fundamentals"},
    {"title": "Embedding Models", "category": "AI Fundamentals"},
    {"title": "Vector Databases", "category": "Infrastructure"}
]

num_chunks = rag.add_documents(
    documents=documents,
    metadatas=metadatas,
    document_type="markdown"
)

print(f"✅ Added {len(documents)} documents")
print(f"📦 Created {num_chunks} chunks")
print(f"📊 Total documents in KB: {rag.get_stats()['total_documents']}")


## Step 4: Query the RAG System

Now let's ask questions and see how the system retrieves relevant context and generates answers.


In [ ]:
# Define test queries
queries = [
    "What are the main capabilities of Large Language Models?",
    "How do embedding models work?",
    "What are some popular vector databases and their features?",
    "Explain the relationship between embeddings and RAG systems."
]

# Process each query
for i, query in enumerate(queries, 1):
    print(f"\n{'='*80}")
    print(f"Query {i}: {query}")
    print('='*80)
    
    # Get RAG response
    response = rag.query(
        query=query,
        top_k=3,  # Retrieve top 3 most relevant chunks
        temperature=0.3  # Lower temperature for more focused answers
    )
    
    # Display answer
    print(f"\n📝 Answer:\n{response.answer}")
    
    # Display sources
    print(f"\n📚 Sources (relevance scores):")
    for idx, (source, score) in enumerate(response.sources, 1):
        print(f"  {idx}. [Score: {score:.3f}] {source[:100]}...")


## Step 5: Advanced Usage - Custom Documents

You can also add your own documents. Let's add a document from a file or URL.


In [ ]:
# Add a custom document about your specific domain
custom_document = """
# Your Custom Knowledge Base

Add your own domain-specific content here. This could be:
- Company documentation
- Technical specifications
- Research papers
- Product information
- FAQ content

The RAG system will chunk this content and make it searchable.
"""

# Add the custom document
rag.add_document(
    document=custom_document,
    metadata={"title": "Custom Content", "category": "User Data"},
    document_type="markdown"
)

# Query about the custom content
custom_query = "What kind of content can I add to my custom knowledge base?"
response = rag.query(custom_query, top_k=2)

print(f"Query: {custom_query}")
print(f"\nAnswer: {response.answer}")
print(f"\n📊 Updated stats: {rag.get_stats()}")


## Step 6: Performance Analysis

Let's analyze the performance of our RAG system.


In [ ]:
import time

# Test retrieval speed
test_queries = [
    "What is machine learning?",
    "How does RAG work?",
    "What are embeddings?",
    "Explain vector databases",
    "What is Claude?"
]

print("⏱️ Performance Testing\n")
times = []

for query in test_queries:
    start = time.time()
    response = rag.query(query, top_k=3)
    elapsed = time.time() - start
    times.append(elapsed)
    print(f"Query: '{query[:30]}...' - Time: {elapsed:.2f}s")

avg_time = np.mean(times)
print(f"\n📊 Average query time: {avg_time:.2f}s")
print(f"📊 Min/Max: {min(times):.2f}s / {max(times):.2f}s")


## Next Steps

Now that you have a working RAG system, you can:

1. **Add more documents**: Load PDFs, web pages, or databases
2. **Optimize chunking**: Experiment with different chunk sizes and overlap
3. **Try different models**: Use Claude 3 Opus for higher quality or Sonnet for balance
4. **Add metadata filtering**: Filter search results by category, date, etc.
5. **Build an API**: Wrap this in a FastAPI or Flask service
6. **Create a UI**: Build a chat interface with Gradio or Streamlit

## Resources

- [Anthropic Documentation](https://docs.anthropic.com/)
- [Voyage AI Documentation](https://docs.voyageai.com/)
- [ChromaDB Documentation](https://docs.trychroma.com/)
- [RAG Best Practices](https://github.com/anthropics/anthropic-cookbook)
